In [19]:
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

In [20]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Hi', 'I am', 'From', 'Singapore', 'Paiseh', 'I', 'Cannot', 'Speak', 'But', 'Machine', 'Learning', 'Can', 'Help', 'Translate'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [21]:
label_map = {label:num for num, label in enumerate(actions)}

In [22]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [23]:
X = np.array(sequences)

In [24]:
y = to_categorical(labels).astype(int)

In [31]:
accuracies = []
f1_scores = []
precisions = []
recalls = []

kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(X):
    
    np.random.shuffle(train_index)
    np.random.shuffle(test_index)
    
    features_train, features_test = X[train_index], X[test_index]
    labels_train, labels_test = y[train_index], y[test_index]
    
    # Construct the required LSTM model.
    model = Sequential() # changed activation to tanh
    model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(30,1662)))
    model.add(LSTM(128, return_sequences=True, activation='tanh'))
    model.add(LSTM(64, return_sequences=False, activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    # Display the success message.
    print("Model Created Successfully!")
    
    # Create an Instance of Early Stopping Callback
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['categorical_accuracy']) 
    
    model_training_history = model.fit(features_train, labels_train, epochs=300, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2, 
                                                     callbacks = [early_stopping_callback])

    # Evaluate the trained model.
    predictions = model.predict(features_test)
    y_true = np.argmax(labels_test, axis=1).tolist()
    y_hat = np.argmax(predictions, axis=1).tolist()
    conf = multilabel_confusion_matrix(y_true,y_hat)
    acc = accuracy_score(y_true, y_hat)
    
    indiv_f1 = []
    indiv_pre = []
    indiv_re = []
    for matrix in conf:
        FP = matrix[0][1]
        FN = matrix[1][0]
        TP = matrix[1][1]
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        indiv_f1.append((2*precision*recall)/(precision+recall))
        indiv_pre.append(precision)
        indiv_re.append(recall)
    f1 = np.nansum(indiv_f1)/len(indiv_f1)
    pre = np.nansum(indiv_pre)/len(indiv_pre)
    re = np.nansum(indiv_re)/len(indiv_re)
    
    accuracies.append(acc)
    f1_scores.append(f1)
    precisions.append(pre)
    recalls.append(re)
    
    print(acc)
    print(f1)

Model Created Successfully!
Epoch 1/300
112/112 [==============================] - 9s 53ms/step - loss: 2.6026 - categorical_accuracy: 0.0804 - val_loss: 2.4225 - val_categorical_accuracy: 0.0714
Epoch 2/300
112/112 [==============================] - 5s 46ms/step - loss: 2.2772 - categorical_accuracy: 0.1295 - val_loss: 2.1451 - val_categorical_accuracy: 0.1429
Epoch 3/300
112/112 [==============================] - 5s 47ms/step - loss: 2.1338 - categorical_accuracy: 0.1518 - val_loss: 2.1308 - val_categorical_accuracy: 0.1696
Epoch 4/300
112/112 [==============================] - 5s 46ms/step - loss: 2.1297 - categorical_accuracy: 0.1629 - val_loss: 2.1082 - val_categorical_accuracy: 0.1250
Epoch 5/300
112/112 [==============================] - 5s 45ms/step - loss: 2.1025 - categorical_accuracy: 0.1362 - val_loss: 2.0453 - val_categorical_accuracy: 0.1339
Epoch 6/300
112/112 [==============================] - 5s 46ms/step - loss: 2.0424 - categorical_accuracy: 0.1674 - val_loss: 2.0282

112/112 [==============================] - 5s 49ms/step - loss: 0.0792 - categorical_accuracy: 0.9911 - val_loss: 0.1818 - val_categorical_accuracy: 0.9554
Epoch 98/300
112/112 [==============================] - 6s 52ms/step - loss: 0.0574 - categorical_accuracy: 0.9955 - val_loss: 0.1515 - val_categorical_accuracy: 0.9732
Epoch 99/300
112/112 [==============================] - 6s 52ms/step - loss: 0.0508 - categorical_accuracy: 0.9933 - val_loss: 0.1547 - val_categorical_accuracy: 0.9732
Epoch 100/300
112/112 [==============================] - 6s 51ms/step - loss: 0.1201 - categorical_accuracy: 0.9754 - val_loss: 0.3646 - val_categorical_accuracy: 0.8839
Epoch 101/300
112/112 [==============================] - 6s 51ms/step - loss: 0.1832 - categorical_accuracy: 0.9442 - val_loss: 0.4253 - val_categorical_accuracy: 0.8929
Epoch 102/300
112/112 [==============================] - 6s 53ms/step - loss: 0.5493 - categorical_accuracy: 0.8326 - val_loss: 0.2224 - val_categorical_accuracy: 0.9

112/112 [==============================] - 6s 51ms/step - loss: 1.3095 - categorical_accuracy: 0.5335 - val_loss: 1.3073 - val_categorical_accuracy: 0.5000
Epoch 18/300
112/112 [==============================] - 6s 50ms/step - loss: 1.3128 - categorical_accuracy: 0.5112 - val_loss: 1.3754 - val_categorical_accuracy: 0.4107
Epoch 19/300
112/112 [==============================] - 6s 49ms/step - loss: 1.2386 - categorical_accuracy: 0.5647 - val_loss: 1.3540 - val_categorical_accuracy: 0.4643
Epoch 20/300
112/112 [==============================] - 6s 52ms/step - loss: 1.1348 - categorical_accuracy: 0.6027 - val_loss: 1.2733 - val_categorical_accuracy: 0.4821
Epoch 21/300
112/112 [==============================] - 5s 49ms/step - loss: 1.0866 - categorical_accuracy: 0.6339 - val_loss: 1.2331 - val_categorical_accuracy: 0.5089
Epoch 22/300
112/112 [==============================] - 6s 49ms/step - loss: 1.1340 - categorical_accuracy: 0.6004 - val_loss: 1.0936 - val_categorical_accuracy: 0.5625

112/112 [==============================] - 6s 53ms/step - loss: 0.8823 - categorical_accuracy: 0.7522 - val_loss: 1.3014 - val_categorical_accuracy: 0.5893
Epoch 23/300
112/112 [==============================] - 6s 53ms/step - loss: 0.8188 - categorical_accuracy: 0.7946 - val_loss: 1.1747 - val_categorical_accuracy: 0.5982
Epoch 24/300
112/112 [==============================] - 6s 51ms/step - loss: 0.8137 - categorical_accuracy: 0.7924 - val_loss: 1.1900 - val_categorical_accuracy: 0.6339
Epoch 25/300
112/112 [==============================] - 6s 52ms/step - loss: 0.7622 - categorical_accuracy: 0.8103 - val_loss: 0.8005 - val_categorical_accuracy: 0.8304
Epoch 26/300
112/112 [==============================] - 6s 54ms/step - loss: 0.6516 - categorical_accuracy: 0.8371 - val_loss: 0.8310 - val_categorical_accuracy: 0.7768
Epoch 27/300
112/112 [==============================] - 6s 54ms/step - loss: 0.6222 - categorical_accuracy: 0.8504 - val_loss: 0.9765 - val_categorical_accuracy: 0.6607

112/112 [==============================] - 6s 54ms/step - loss: 0.2273 - categorical_accuracy: 0.9353 - val_loss: 0.5027 - val_categorical_accuracy: 0.8571
Epoch 119/300
112/112 [==============================] - 6s 53ms/step - loss: 0.1419 - categorical_accuracy: 0.9487 - val_loss: 0.1717 - val_categorical_accuracy: 0.9643
Epoch 120/300
112/112 [==============================] - 6s 54ms/step - loss: 0.0987 - categorical_accuracy: 0.9665 - val_loss: 0.2007 - val_categorical_accuracy: 0.9554
Epoch 121/300
112/112 [==============================] - 6s 54ms/step - loss: 0.1539 - categorical_accuracy: 0.9531 - val_loss: 0.2005 - val_categorical_accuracy: 0.9464
Epoch 122/300
112/112 [==============================] - 6s 51ms/step - loss: 0.1788 - categorical_accuracy: 0.9420 - val_loss: 0.5156 - val_categorical_accuracy: 0.8393
Epoch 123/300
112/112 [==============================] - 6s 52ms/step - loss: 0.1126 - categorical_accuracy: 0.9688 - val_loss: 0.4775 - val_categorical_accuracy: 0

Epoch 11/300
112/112 [==============================] - 5s 48ms/step - loss: 1.7924 - categorical_accuracy: 0.3192 - val_loss: 1.7984 - val_categorical_accuracy: 0.3214
Epoch 12/300
112/112 [==============================] - 5s 47ms/step - loss: 1.6878 - categorical_accuracy: 0.3482 - val_loss: 1.7111 - val_categorical_accuracy: 0.3304
Epoch 13/300
112/112 [==============================] - 5s 47ms/step - loss: 1.6105 - categorical_accuracy: 0.4040 - val_loss: 1.7548 - val_categorical_accuracy: 0.3571
Epoch 14/300
112/112 [==============================] - 5s 47ms/step - loss: 1.5908 - categorical_accuracy: 0.3772 - val_loss: 1.7887 - val_categorical_accuracy: 0.3571
Epoch 15/300
112/112 [==============================] - 5s 47ms/step - loss: 1.6206 - categorical_accuracy: 0.3638 - val_loss: 1.4856 - val_categorical_accuracy: 0.5000
Epoch 16/300
112/112 [==============================] - 5s 48ms/step - loss: 1.5165 - categorical_accuracy: 0.4263 - val_loss: 1.4505 - val_categorical_acc

112/112 [==============================] - 6s 52ms/step - loss: 0.1696 - categorical_accuracy: 0.9330 - val_loss: 0.2838 - val_categorical_accuracy: 0.9018
Epoch 108/300
112/112 [==============================] - 6s 54ms/step - loss: 0.3387 - categorical_accuracy: 0.8817 - val_loss: 0.4224 - val_categorical_accuracy: 0.8571
Epoch 109/300
112/112 [==============================] - 7s 65ms/step - loss: 0.1854 - categorical_accuracy: 0.9487 - val_loss: 0.1949 - val_categorical_accuracy: 0.9375
Epoch 110/300
112/112 [==============================] - 7s 66ms/step - loss: 0.1121 - categorical_accuracy: 0.9665 - val_loss: 0.1105 - val_categorical_accuracy: 0.9643
Epoch 111/300
112/112 [==============================] - 7s 65ms/step - loss: 0.1000 - categorical_accuracy: 0.9732 - val_loss: 0.1935 - val_categorical_accuracy: 0.9464
Epoch 112/300
112/112 [==============================] - 8s 69ms/step - loss: 0.4717 - categorical_accuracy: 0.8549 - val_loss: 0.4886 - val_categorical_accuracy: 0

112/112 [==============================] - 9s 58ms/step - loss: 2.6477 - categorical_accuracy: 0.0781 - val_loss: 2.6428 - val_categorical_accuracy: 0.0714
Epoch 2/300
112/112 [==============================] - 6s 50ms/step - loss: 2.4684 - categorical_accuracy: 0.1696 - val_loss: 2.3855 - val_categorical_accuracy: 0.1696
Epoch 3/300
112/112 [==============================] - 6s 52ms/step - loss: 2.2699 - categorical_accuracy: 0.1652 - val_loss: 2.2037 - val_categorical_accuracy: 0.1964
Epoch 4/300
112/112 [==============================] - 6s 52ms/step - loss: 2.1758 - categorical_accuracy: 0.1629 - val_loss: 2.1703 - val_categorical_accuracy: 0.0536
Epoch 5/300
112/112 [==============================] - 6s 53ms/step - loss: 2.1224 - categorical_accuracy: 0.1741 - val_loss: 2.1399 - val_categorical_accuracy: 0.1071
Epoch 6/300
112/112 [==============================] - 6s 56ms/step - loss: 2.0683 - categorical_accuracy: 0.2009 - val_loss: 2.0514 - val_categorical_accuracy: 0.2500
Epoc

112/112 [==============================] - 7s 59ms/step - loss: 0.3546 - categorical_accuracy: 0.8906 - val_loss: 0.3394 - val_categorical_accuracy: 0.8929
Epoch 98/300
112/112 [==============================] - 6s 57ms/step - loss: 0.2547 - categorical_accuracy: 0.9219 - val_loss: 0.5106 - val_categorical_accuracy: 0.8125
Epoch 99/300
112/112 [==============================] - 6s 53ms/step - loss: 0.3036 - categorical_accuracy: 0.9219 - val_loss: 0.4909 - val_categorical_accuracy: 0.8482
Epoch 100/300
112/112 [==============================] - 6s 57ms/step - loss: 0.2009 - categorical_accuracy: 0.9397 - val_loss: 0.1247 - val_categorical_accuracy: 0.9911
Epoch 101/300
112/112 [==============================] - 6s 56ms/step - loss: 0.1645 - categorical_accuracy: 0.9598 - val_loss: 0.3638 - val_categorical_accuracy: 0.8929
Epoch 102/300
112/112 [==============================] - 6s 53ms/step - loss: 0.0796 - categorical_accuracy: 0.9933 - val_loss: 0.0790 - val_categorical_accuracy: 1.0

In [32]:
print("Accuracies:", accuracies)
print("F1 Scores:", f1_scores)
print("Precisions:", precisions)
print("Recalls:", recalls)

Accuracies: [0.9285714285714286, 0.9714285714285714, 0.9357142857142857, 0.9928571428571429, 0.9785714285714285]
F1 Scores: [0.9237074110041039, 0.9776475155279505, 0.9375783033391729, 0.9933834586466166, 0.9793083900226758]
Precisions: [0.9261028653885797, 0.9785943223443223, 0.9423849959564246, 0.9928571428571429, 0.9825174825174825]
Recalls: [0.9300804093567253, 0.9781364468864469, 0.9422902494331067, 0.9945054945054945, 0.9792207792207792]


In [33]:
print("Mean Accuracy:", np.nansum(accuracies)/len(accuracies))
print("Mean F1 Score:", np.nansum(f1_scores)/len(f1_scores))
print("Mean Precision:", np.nansum(precisions)/len(precisions))
print("Mean Recall:", np.nansum(recalls)/len(recalls))

Mean Accuracy: 0.9614285714285714
Mean F1 Score: 0.962325015708104
Mean Precision: 0.9644913618127904
Mean Recall: 0.9648466758805105
